In [1]:
import os
os.chdir(os.getcwd().replace('/Code', ''))
print(os.getcwd())

/home/id1019/Documents/PFE/GITHUB


In [2]:
import gzip
import pickle as pkl
import pandas as pd
import langdetect
from textblob import TextBlob
from multiprocessing import Pool
from functools import reduce
from collections import Counter
from googletrans import Translator


## Récupération de la matrice précédente

In [4]:
with gzip.open('Matrices/CVs1_suppressionCV.pkl.gz', 'rb') as f:
    CVs = pkl.load(f)
print(CVs.shape)
CVs.head(1)

(8212, 3)


,id_CV,text_no_x,len_cv_initial
id_CV,,,
5711,5711,Jaouadi – Junior Data Scientist\n\n\n Jaoua...,4477


## Traduction en anglais

In [5]:
CVs['traduction'] = [ None for cv in CVs.index ]

In [6]:
print('Il faut traduire', len([i for i in CVs.index  if CVs.loc[i]['traduction'] == None]), 'cv.')

Il faut traduire 8212 cv.


In [101]:
%%time
#45 minutes si la traduction n'a pas déjà été lancée
try :
    with gzip.open('Matrices/CVs2_traductionALL_srcfr.pkl.gz', 'rb') as f:
        CVs = pkl.load(f)
    print(CVs.shape)
    print('Il reste', len([i for i in CVs.index  if CVs.loc[i]['traduction'] == None]), 'cv à traduire.')
except:
    print("premiere traduction")

    
liste_CV_id_a_traduire= [[CVs.loc[cv]['text_no_x'], cv] for cv in CVs.index if CVs.loc[cv]['traduction'] == None]
print(len(liste_CV_id_a_traduire))    
    
def etape_map(CV_and_id):
    CV = CV_and_id[0]
    id_CV = CV_and_id[1]
    CV_traduit = None
    try :
        blob = TextBlob(CV )
        CV_traduit = str(blob.translate(from_lang='fr', to='en'))
    except:
        try :
            translator = Translator()
            CV_traduit = translator.translate( CV,src='fr', dest='en').text
        except :
            try:
                #Découpage du cv en 4 parties
                lignes = CV.split('\n')
                nb_lignes = len(lignes)
                size = int(np.floor(nb_lignes/4))
                c1 = "\n".join(CV.split('\n')[0:size])
                c1_traduit = translator.translate(c1 ,src='fr', dest='en').text
                c2 = "\n".join(CV.split('\n')[size:2*size])
                c2_traduit = translator.translate(c2 ,src='fr', dest='en').text
                c3 = "\n".join(CV.split('\n')[2*size:3*size])
                c3_traduit = translator.translate(c3 ,src='fr', dest='en').text
                c4 = "\n".join(CV.split('\n')[3*size:])
                c4_traduit = translator.translate(c4 ,src='fr', dest='en').text
                cr = "\n".join([c1,c2,c3,c4])
                CV_traduit = "\n".join([c1_traduit, c2_traduit, c3_traduit, c4_traduit])
            except:
                print("la traduction du cv n°",id_CV,"n'a pas fonctionnée")
    return ([id_CV], [CV_traduit])

def etape_reduce(a, b):
    a[0].append(b[0][0])
    a[1].append(b[1][0])
    return a
    
with Pool() as pool:
    m = pool.map(etape_map, liste_CV_id_a_traduire) 
reduce_traduction = reduce(etape_reduce, m)

for i in range(len(reduce_traduction[0])):
    id_cv = reduce_traduction[0][i]
    CVs.loc[id_cv,'traduction'] = reduce_traduction[1][i]

        
for CV in CVs.index:
    if CVs.loc[CV,'traduction']!=None:
        CVs.loc[CV,'traduction'] = CVs.loc[CV]['traduction'].replace("C ++", 'C++')
        CVs.loc[CV,'traduction'] = CVs.loc[CV]['traduction'].replace("c ++", 'C++')
        CVs.loc[CV,'traduction'] = CVs.loc[CV]['traduction'].replace("C #", 'C#')
        CVs.loc[CV,'traduction'] = CVs.loc[CV]['traduction'].replace("c #", 'C++')
        
        
with gzip.open('Matrices/CVs2_traductionALL_srcfr.pkl.gz', 'wb') as f:
    pkl.dump(CVs, f)

print(CVs.shape)
print('Il reste', len([i for i in CVs.index  if CVs.loc[i]['traduction'] == None]), 'cv à traduire.')

(8212, 4)
Il reste 7 cv à traduire.
7
la traduction du cv n° 1975 n'a pas fonctionné
la traduction du cv n° 7521 n'a pas fonctionné
la traduction du cv n° 2046 n'a pas fonctionné
la traduction du cv n° 156 n'a pas fonctionné
la traduction du cv n° 3997 n'a pas fonctionné
la traduction du cv n° 7111 n'a pas fonctionné
la traduction du cv n° 3186 n'a pas fonctionné
(8212, 4)
Il reste 7 cv à traduire.
CPU times: user 32.2 s, sys: 204 ms, total: 32.4 s
Wall time: 36.9 s
